# Assignment 7: Decision Tree Classification

## Dataset: Heart Disease Prediction

**Objective:** Apply Decision Tree Classification to predict heart disease.

**Topics Covered:**
- Data Preparation and EDA
- Feature Engineering
- Decision Tree Classification
- Hyperparameter Tuning
- Model Evaluation

---
## Step 1: Import Libraries and Load Data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder

# Load the dataset (second sheet contains the data)
df = pd.read_excel('heart_disease.xlsx', sheet_name=1)

print("Dataset loaded successfully!")
print("Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

---
## Step 2: Exploratory Data Analysis (EDA)

In [ ]:
# Check data types and info
print("=== Dataset Info ===")
print(df.dtypes)
print("\n=== Statistical Summary ===")
df.describe()

In [ ]:
# Check for missing values
print("=== Missing Values ===")
missing = df.isnull().sum()
print(missing)
print("\nTotal missing:", missing.sum())

In [ ]:
# Check unique values in categorical columns
print("=== Unique Values in Categorical Columns ===")
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

for col in categorical_cols:
    print(col + ":", df[col].unique())

In [ ]:
# Target variable distribution
print("=== Target Variable (num) Distribution ===")
target_counts = df['num'].value_counts()
print(target_counts)

# Convert to binary (0 = no disease, 1 = disease)
# Original: 0, 1, 2, 3, 4 (stages of heart disease)
# Binary: 0 = no disease, 1 = has disease
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)

print("\nAfter converting to binary:")
print(df['target'].value_counts())

In [ ]:
# Visualize target distribution
plt.figure(figsize=(8, 5))
colors = ['green', 'red']
target_counts = df['target'].value_counts()
plt.bar(['No Disease', 'Has Disease'], target_counts.values, color=colors)
plt.xlabel('Heart Disease Status')
plt.ylabel('Count')
plt.title('Distribution of Heart Disease')

# Add count labels
for i in range(len(target_counts)):
    plt.text(i, target_counts.values[i] + 10, str(target_counts.values[i]), ha='center')

plt.savefig('target_distribution.png')
plt.show()

In [ ]:
# Histograms for numerical columns
numerical_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(len(numerical_cols)):
    col = numerical_cols[i]
    axes[i].hist(df[col], bins=30, color='steelblue', edgecolor='black')
    axes[i].set_title('Distribution of ' + col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

# Hide last subplot
axes[5].axis('off')

plt.tight_layout()
plt.savefig('histograms.png')
plt.show()

In [ ]:
# Boxplots to check for outliers
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(len(numerical_cols)):
    col = numerical_cols[i]
    axes[i].boxplot(df[col].dropna())
    axes[i].set_title('Boxplot of ' + col)
    axes[i].set_ylabel(col)

axes[5].axis('off')

plt.tight_layout()
plt.savefig('boxplots.png')
plt.show()

---
## Step 3: Feature Engineering

In [ ]:
# Handle missing values
print("=== Handling Missing Values ===")

# Fill numerical missing values with median
for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print("Filled", col, "with median:", median_val)

# Fill categorical missing values with mode
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()[0]
        df[col] = df[col].fillna(mode_val)
        print("Filled", col, "with mode:", mode_val)

print("\nRemaining missing values:", df.isnull().sum().sum())

In [ ]:
# Encode categorical variables using Label Encoding
print("=== Label Encoding Categorical Variables ===")

# Create a copy for encoding
df_encoded = df.copy()

# Apply label encoding to categorical columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le
    print(col + ":")
    print("  Original:", list(le.classes_))
    print("  Encoded:", list(range(len(le.classes_))))
    print()

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 10))

# Select only numerical columns for correlation
numeric_df = df_encoded[numerical_cols + categorical_cols + ['target']]
correlation = numeric_df.corr()

sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.show()

---
## Step 4: Decision Tree Classification

In [ ]:
# Prepare features and target
print("=== Preparing Data for Model ===")

# Features (X) - all columns except 'num' and 'target'
feature_cols = numerical_cols + categorical_cols
X = df_encoded[feature_cols]

# Target (y)
y = df_encoded['target']

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("\nFeature columns:", feature_cols)

In [ ]:
# Split data into training and testing sets (80-20 split)
print("=== Train-Test Split ===")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))
print("\nTraining percentage:", round(len(X_train)/len(X)*100, 1), "%")
print("Testing percentage:", round(len(X_test)/len(X)*100, 1), "%")

In [ ]:
# Create and train Decision Tree model
print("=== Training Decision Tree Model ===")

# Create model with default parameters
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

print("Model trained successfully!")
print("\nModel parameters:")
print("  Max depth:", dt_model.max_depth)
print("  Min samples split:", dt_model.min_samples_split)
print("  Criterion:", dt_model.criterion)

In [ ]:
# Make predictions
y_pred = dt_model.predict(X_test)
y_pred_proba = dt_model.predict_proba(X_test)[:, 1]

print("Predictions made on test set")
print("\nSample predictions (first 10):")
for i in range(10):
    print("Actual:", y_test.iloc[i], "| Predicted:", y_pred[i])

In [ ]:
# Evaluate model performance
print("=== Model Evaluation (Default Parameters) ===")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1-Score:", round(f1, 4))
print("ROC-AUC:", round(roc_auc, 4))

In [ ]:
# Confusion Matrix
print("=== Confusion Matrix ===")

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Disease', 'Has Disease'],
            yticklabels=['No Disease', 'Has Disease'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.show()

print("\nInterpretation:")
print("True Negatives:", cm[0][0])
print("False Positives:", cm[0][1])
print("False Negatives:", cm[1][0])
print("True Positives:", cm[1][1])

In [ ]:
# Classification Report
print("=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=['No Disease', 'Has Disease']))

---
## Step 5: Hyperparameter Tuning

In [ ]:
# Hyperparameter Tuning
print("=== Hyperparameter Tuning ===")
print("")
print("Testing different values for:")
print("- max_depth: Maximum depth of tree")
print("- min_samples_split: Minimum samples to split a node")
print("- criterion: 'gini' or 'entropy'")
print("")

# Store results
results = []

# Try different combinations
max_depths = [3, 5, 7, 10, None]
min_samples_splits = [2, 5, 10]
criterions = ['gini', 'entropy']

best_accuracy = 0
best_params = {}

for depth in max_depths:
    for min_split in min_samples_splits:
        for criterion in criterions:
            # Create and train model
            model = DecisionTreeClassifier(
                max_depth=depth,
                min_samples_split=min_split,
                criterion=criterion,
                random_state=42
            )
            model.fit(X_train, y_train)
            
            # Evaluate
            y_pred_temp = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred_temp)
            
            # Store result
            results.append({
                'max_depth': depth,
                'min_samples_split': min_split,
                'criterion': criterion,
                'accuracy': acc
            })
            
            # Update best
            if acc > best_accuracy:
                best_accuracy = acc
                best_params = {
                    'max_depth': depth,
                    'min_samples_split': min_split,
                    'criterion': criterion
                }

print("Tuning complete!")
print("\nBest parameters found:")
print("  max_depth:", best_params['max_depth'])
print("  min_samples_split:", best_params['min_samples_split'])
print("  criterion:", best_params['criterion'])
print("\nBest accuracy:", round(best_accuracy, 4))

In [ ]:
# Train model with best parameters
print("=== Training Model with Best Parameters ===")

best_model = DecisionTreeClassifier(
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    criterion=best_params['criterion'],
    random_state=42
)
best_model.fit(X_train, y_train)

# Evaluate
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, y_pred_proba_best)

print("\n=== Tuned Model Performance ===")
print("Accuracy:", round(accuracy_best, 4))
print("Precision:", round(precision_best, 4))
print("Recall:", round(recall_best, 4))
print("F1-Score:", round(f1_best, 4))
print("ROC-AUC:", round(roc_auc_best, 4))

---
## Step 6: Model Visualization and Analysis

In [ ]:
# Visualize the Decision Tree
print("=== Decision Tree Visualization ===")

plt.figure(figsize=(20, 10))
plot_tree(best_model, 
          feature_names=feature_cols,
          class_names=['No Disease', 'Has Disease'],
          filled=True,
          rounded=True,
          fontsize=10,
          max_depth=3)  # Show only first 3 levels for clarity
plt.title('Decision Tree (First 3 Levels)')
plt.tight_layout()
plt.savefig('decision_tree.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Feature Importance
print("=== Feature Importance ===")

# Get feature importances
importances = best_model.feature_importances_

# Create dataframe for visualization
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': importances
})

# Sort by importance
importance_df = importance_df.sort_values('Importance', ascending=True)

# Plot
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='steelblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance in Decision Tree')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.show()

print("\nTop 5 Most Important Features:")
top_features = importance_df.sort_values('Importance', ascending=False).head(5)
for i in range(len(top_features)):
    row = top_features.iloc[i]
    print(str(i+1) + ".", row['Feature'], "-", round(row['Importance'], 4))

In [ ]:
# ROC Curve
print("=== ROC Curve ===")

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_best)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label='ROC Curve (AUC = ' + str(round(roc_auc_best, 4)) + ')')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True)
plt.savefig('roc_curve.png')
plt.show()

---
## Interview Questions

### 1. Common Hyperparameters of Decision Trees

| Hyperparameter | Description | Effect |
|----------------|-------------|--------|
| **max_depth** | Maximum depth of tree | Higher = more complex, may overfit |
| **min_samples_split** | Minimum samples to split a node | Higher = less complex, prevents overfitting |
| **min_samples_leaf** | Minimum samples in leaf node | Higher = smoother decision boundary |
| **criterion** | Splitting criterion (gini/entropy) | Gini is faster, entropy may give slightly better results |
| **max_features** | Number of features to consider | Lower = more randomness, reduces overfitting |

### 2. Label Encoding vs One-Hot Encoding

| Aspect | Label Encoding | One-Hot Encoding |
|--------|---------------|------------------|
| **Method** | Assigns numbers (0,1,2...) | Creates binary columns |
| **Columns** | Same number of columns | Creates many new columns |
| **Assumption** | Implies ordinal relationship | No ordinal assumption |
| **Best for** | Ordinal categories, Tree models | Nominal categories, Linear models |
| **Example** | Low=0, Medium=1, High=2 | Color_Red=1, Color_Blue=0 |

---
## Summary

In this assignment, we:

1. **Loaded and explored** the heart disease dataset
2. **Preprocessed data** by handling missing values and encoding categorical variables
3. **Built a Decision Tree** classifier with 80-20 train-test split
4. **Tuned hyperparameters** to find the best model configuration
5. **Evaluated the model** using accuracy, precision, recall, F1-score, and ROC-AUC
6. **Visualized** the decision tree and feature importance